In [1]:
import requests
import bs4 as bs
import pandas as pd
from datetime import date
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [2]:
#input date to get fixtures from
matchDay = input('Input Match Date (YYYY/MM/DD):')

#assign vaiable to url
url = 'https://ng.soccerway.com/matches/'+matchDay

#assign 'headers' device and browser illusion
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
           'AppleWebKit/537.36 (KHTML, like Gecko) '\
           'Chrome/75.0.3770.80 Safari/537.36'}

#download html of site
page = requests.get(url, headers=headers)

#create object from downloaded html
soup = bs.BeautifulSoup(page.text, 'html.parser')

#find the 'a' tags ins all 'h2' tags
links = [l.find('a') for l in soup.find_all('h2')]

#remove all none items
links = [l for l in links if l != None]
#get all hrefs
links = [l.get('href') for l in links]

#get all items bearing 'national/'
links = [l for l in links if 'national/' in l]

#convert href to links
links = ['https://ng.soccerway.com/'+l for l in links]

#links = list of all competition links in matchDay's page

Input Match Date (YYYY/MM/DD):2023/03/24


In [3]:
#create webdriver object for Edge Browser
driver = webdriver.Edge()

#maximize window
driver.maximize_window()

#sleep for 2s
time.sleep(2)

# for loop begins here

In [4]:
##to be loop for all competiton links
#open Edge browser
driver.get(links[0])

In [11]:
#find the element button for 'by date' (using xpath) and click on it
driver.find_element('xpath', '//*[@id="page_competition_1_block_competition_matches_summary_9_subnav"]/li[2]').click()

#wait for 20s
time.sleep(6)


In [12]:
#find webdriver object with class name 'table-container.redesign'
tableBox= driver.find_element(By.CLASS_NAME, "table-container.redesign")

#find elements all 'a' tag element within 
tbLinks = tableBox.find_elements(By.TAG_NAME, 'a')

#append attribute of webdriver object into fixtureLinks
fixtureLinks = []
for item in tbLinks:
    fixtureLinks.append(item.get_attribute('outerHTML'))
    
#fixturelinks = list of all hrefs 

In [13]:
#filter fixtureLinks with '/matches/'
fixtureLinks = [l for l in fixtureLinks if '/matches/'+matchDay in l]

#pick href value 
fixtureLinks = [l.split('"')[1] for l in fixtureLinks]

#remove hrefs bearing 'event'
fixtureLinks = [l for l in fixtureLinks if 'event' not in l]

#frixtureLinks = list of all matchDay fixture pages

In [14]:
fixtureLinks

['/matches/2023/03/24/europe/european-championship-qualification/bulgaria/montenegro/3972724/',
 '/matches/2023/03/24/europe/european-championship-qualification/france/netherlands/3972623/',
 '/matches/2023/03/24/europe/european-championship-qualification/gibraltar/greece/3972624/',
 '/matches/2023/03/24/europe/european-championship-qualification/czech-republic/poland/3972684/',
 '/matches/2023/03/24/europe/european-championship-qualification/moldova/faroe-islands/3972685/',
 '/matches/2023/03/24/europe/european-championship-qualification/sweden/belgium/3972704/',
 '/matches/2023/03/24/europe/european-championship-qualification/austria/azerbaijan/3972705/',
 '/matches/2023/03/24/europe/european-championship-qualification/serbia/lithuania/3972725/']

In [15]:
#convert to web link
fixtureLinks = ['https://ng.soccerway.com'+l for l in fixtureLinks]

#fixtureLinks = list of weblinks for matchDay's matches

In [141]:
fixtureLinks[2]

'https://ng.soccerway.com/matches/2023/03/24/europe/european-championship-qualification/gibraltar/greece/3972624/'

In [142]:
#to be looped for all fixtureLinks
driver.get(fixtureLinks[2])

In [143]:
#find elements with class name = team-title
matches = driver.find_elements(By.CLASS_NAME, "team-title")

In [144]:
#get html attribute of found element for home team
homeLink = matches[0].get_attribute('outerHTML').split('"')[3]

#get html attribute of found element for away team
awayLink = matches[1].get_attribute('outerHTML').split('"')[3]

#convert to matches pagelink of home and away teams
homeLink = 'https://ng.soccerway.com'+homeLink+'/matches/'
awayLink = 'https://ng.soccerway.com'+awayLink+'/matches/'

print (homeLink)
print (awayLink)

https://ng.soccerway.com/teams/gibraltar/gibraltar/7921//matches/
https://ng.soccerway.com/teams/greece/greece/1091//matches/


In [145]:
#get html of home matches page
homeLinkH = requests.get(homeLink, headers=headers)

#read the first table of the html
homeDF = pd.read_html(homeLinkH.text)[0]

#rename columns
homeDF.rename(columns={'Home team':'Score', 'Score/Time': 'Away','Outcome': 'Home'}, inplace=True)

#rewrite matchDay from yyyy/mm/dd to dd/mm/yy
dayMatch = matchDay[8:]+matchDay[4:8]+matchDay[2:4]

#create homeDF1 which is all rows bearing dayMatch as its Date data
homeDF1 = homeDF[homeDF['Date'] == dayMatch]

#rertrieve row index for matchDay and assign to variable date_indexH
date_indexH = int(list(homeDF1['Date'].index)[0])

#get the competiton data of the first row of homeDF1 assign to variable called compName
compName = homeDF1.loc[date_indexH,'Competition']

#home & away team names
homeName = homeDF1.loc[date_indexH,'Home']
awayName = homeDF1.loc[date_indexH,'Away']

#cleaning homeDF

#drop all rows except one with Competition column data = compName 
homeDF = homeDF[homeDF['Competition'] == compName]

#reset index location
homeDF = homeDF.reset_index(drop=True)

#create homeDF2 which is all rows bearing dayMatch as its Date data
homeDF2 = homeDF[homeDF['Date'] == dayMatch]

#rertrieve row index for matchDay and assign to variable date_indexH2
date_indexH2 = int(list(homeDF2['Date'].index)[0])

#drop all rows including and after matchDay
homeDF = homeDF.iloc[:date_indexH2]

#drop all columns except specificied
homeDF = homeDF[['Home', 'Score', 'Away']]

#calculate points and append as column
score_list = homeDF['Score'].values.tolist()
score_list1 = []
for a in score_list:
    try:
        eval(a)
    except:
        score_list1.append(None)
    else:
        score_list1.append(eval(a))
homeDF['Points'] = score_list1

#reverse rows in homeDF
homeDF = homeDF.iloc[::-1]

#reset index location
homeDF = homeDF.reset_index(drop=True)

#DF for last 5 matches where home team played at home
homeDF_home = homeDF[homeDF['Home']==homeName].head(5)

#DF for last 5 macthes for home
homeDF_Gen = homeDF.head(5)

#DF for last 5 matches where home team played at away
homeDF_away = homeDF[homeDF['Away']==homeName].head(5)


#homeName = name of home team
#awayName = name of away team
#homeDF = DF for all matches for home team
#compName = Competiton name home POV
#homeDF_home = home's home analysis
#homeDF_away = home's away analysis

In [62]:
#get html of away matches page
awayLinkH = requests.get(awayLink, headers=headers)

#read the first table of the html
awayDF = pd.read_html(awayLinkH.text)[0]

#rename columns
awayDF.rename(columns={'Home team':'Score', 'Score/Time': 'Away','Outcome': 'Home'}, inplace=True)

#create awayDF1 which is all rows bearing dayMatch as its Date data
awayDF1 = awayDF[awayDF['Date'] == dayMatch]

#get row number of matchDay
date_indexA = int(list(awayDF1['Date'].index)[0])

#get the competiton data of the first row of homeDF1 assign to variable called compName
compName2 = awayDF1.loc[date_indexA,'Competition']

#drop all rows except one with Competition column data = compName 
awayDF = awayDF[awayDF['Competition'] == compName2]

#reset index
awayDF = awayDF.reset_index(drop=True)

#create awayDF2 which is all rows bearing dayMatch as its Date data
awayDF2 = awayDF[awayDF['Date'] == dayMatch]

#get row number of matchDay
date_indexA2 = int(list(awayDF2['Date'].index)[0])

#drop all rows including and after macthDay
awayDF = awayDF.iloc[:date_indexA2]  

#drop all columns except specificied
awayDF = awayDF[['Home', 'Score', 'Away']]

#calculate points and append as column
score_listA = awayDF['Score'].values.tolist()
score_listA1 = []
for a in score_listA:
    try:
        eval(a)
    except:
        score_listA1.append(None)
    else:
        score_listA1.append(eval(a))
awayDF['Points'] = score_listA1

#reverse rows in awayDF
awayDF = awayDF.iloc[::-1]

#DF for last 5 matches where away team played at home
awayDF_home = awayDF[awayDF['Home']==awayName].head(5)

#DF for last 5 matches where away team played at away
awayDF_away = awayDF[awayDF['Away']==awayName].head(5)


#awayName = name of away team
#compName2 = competition name away POV
#awayDF = DF for matches played by away team
#awayDF_home = DF for away's home analysis
#awayDF_away = DF for away's away analysis



In [ ]:
#evaluate home team's home form 

#convert 'Points' homeDF_home columns to a list
HPointsList = homeDF_home['Points'].values.tolist()

#counter
num001 = 0
#iterate across list, and change point data into correct point value
for point in HPointsList:
    if point > 0:
        HPointsList[num001] = 1
    elif point < 0:
        HPointsList[num001] = 0
    elif point == 0:
        HPointsList[num001] = 0.5
    else:
        pass
    num001+=1
#calculate homeform value
home_form_value = sum(HPointsList)

#evaluate away team's away form

#convert 'Points' awayDF_away columns to a list
APointsList = awayDF_away['Points'].values.tolist()

#counter
num003 = 0
#iterate across list, and change point data into correct point value
for point in APointsList:
    if point > 0:
        APointsList[num003] = 1
    elif point < 0:
        APointsList[num003] = 0
    elif point == 0:
        APointsList[num003] = 0.5
    else:
        pass
    num003+=1
#calculate awayform value
away_form_value = sum(APointsList)

#evaluate home team's general form

#convert 'Points' homeDF_Gen columns to a list
HGPointsList = homeDF_Gen['Points'].values.tolist()

#convert 'Home' homeDF_Gen columns to a list
homeTeamList = homeDF_Gen['Home'].values.tolist()

#counter
num002 = 0
#iterate across list, and change point data into correct point value
for point in HGPointsList:
    if homeTeamList[num002] == homeName:
        if point > 0:
            HGPointsList[num002] = 1
        elif point < 0:
            HGPointsList[num002] = 0
        elif point == 0:
            HGPointsList[num002] = 0.5
        else:
            pass
    else:
        if point > 0:
            HGPointsList[num002] = 0
        elif point < 0:
            HGPointsList[num002] = 1
        elif point == 0:
            HGPointsList[num002] = 0.5
        else:
            pass
    num002+=1
#calculate homeGen value
home_Gen_value = sum(HGPointsList)


In [ ]:
#HOME TEAM GOALS

#convert homeDF_home 'Score' column into list
homeGoalList = homeDF_home['Score'].values.tolist()

#check score lines in above homeGoallist and append 1 point (homeGoalF) to scorelines where home scored
homeGoalF = []
for goal in homeGoalList:
    if int(goal[0]) == 0:
        pass
    else:
        homeGoalF.append(1)
#sum the appended points to get home_GF
home_GF = sum(homeGoalF)

#check score lines in above homeGoalList and 
#append 1 point (homeGoalA) to scorelines where home conceded
homeGoalA = []
for goal in homeGoalList[:2]:
    if int(goal[4]) == 0:
        pass
    else:
        homeGoalA.append(1)
#sum the appended points to get home_GA
home_GA = sum(homeGoalA)

#AWAY TEAM GOALS

#convert awayDF_away 'Score' column into list
awayGoalList = awayDF_away['Score'].values.tolist()

#check score lines in above awayGoallist and 
#append 1 point (awayGoalA) to scorelines where away conceded
awayGoalA = []
for goal in awayGoalList:
    if int(goal[0]) == 0:
        pass
    else:
        awayGoalA.append(1)
#sum the appended points to get away_GA
away_GA = sum(awayGoalA)

#check score lines in above awayGoallist and 
#append 1 point (awayGoalF) to scorelines where away scored
awayGoalF = []
for goal in awayGoalList[:2]:
    if int(goal[4]) == 0:
        pass
    else:
        awayGoalF.append(1)
#sum the appended points to get away_GF
away_GF = sum(awayGoalF)

# h2h

In [131]:
#link to h2h
dink = fixtureLinks[2]+'head2head'
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
           'AppleWebKit/537.36 (KHTML, like Gecko) '\
           'Chrome/75.0.3770.80 Safari/537.36'}

#request h2h page
pgaa = requests.get(dink, headers=headers)



In [132]:
#read html page using pandas call it jon
jon = pd.read_html(pgaa.text)[0]

##the table is arranged in anti-chronology, the recent data is on top
##and the  match to be played is the first row
##which want two most recent h2h data

#reverse rows in jon
jon = jon.iloc[::-1]

#reset index
jon = jon.reset_index(drop=True)

#create jon1 which is all rows bearing dayMatch as its Date data
jon1 = jon[jon['Date'] == dayMatch]

#get row number of matchDay
date_indexJ = int(list(jon1['Date'].index)[0])

#drop all rows including and after the specified index row
jon = jon.iloc[:date_indexJ]

#drop all columns except specificied
jon = jon[['Home team', 'Score/Time', 'Away team']]

#rename columns
jon.rename(columns={'Home team':'Home', 'Score/Time': 'Score','Away team': 'Away'}, inplace=True)

#drop all rows except last 2 rows
jon = jon.tail(2)

In [147]:
try:
    jon.loc[0, 'Home']
except:
    x = [0, 'NaN']
else:
    if jon.loc[0,'Home'] == jon.loc[jon.shape[0] - 1,'Home']:
        result=[]
        num002 = 0
        for a in jon['Score']:
            try:
                eval(a)
            except:
                result.append(-1)
            else:                
                gd = int(eval(jon.loc[num002,'Score']))
                result.append(gd)               
            num002+=1
        num003 = 0
        for b in result:
            if b > 0:
                result[num003] = 1
            elif b < 0:
                result[num003] = 0
            else:
                result[num003] = 0.5
            num003+=1
        hn = jon.loc[0, 'Home']
        result = sum(result)

    else:
        result=[]
        try:
            eval(jon.loc[0,'Score'])
        except:
            result.append(-1)
        else:
            gd = int(eval(jon.loc[0, 'Score']))
            result.append(gd)
        try:
            eval(jon.loc[1,'Score'])
        except:
            result.append(-1)
        else:
            gd = int(eval(jon.loc[1, 'Score'][::-1]))
            result.append(gd)
        num004 = 0           
        for c in result:
            if c > 0:
                result[num004] = 1
            elif c < 0:
                result[num004] = 0
            else:
                result[num004] = 0.5
            num004+=1
        result = sum(result)
        hn = jon.loc[0, 'Home']

    if hn in homeName:
        pass
    else:
        result = 2 - result

    h2hgoalist = []
    num005 = 0
    for a in jon['Score']:
        x = a.replace('-','+')
        if int(eval(x)) >= 2:
            h2hgoalist.append(1)
        else:
            h2hgoalist.append(0)
        num005+=1
    h2h_goalist = sum(h2hgoalist)  

